In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

class Trade:
    def __init__(self, buy_price, buy_index, buy_date):
        self.buy_price = buy_price
        self.buy_index = buy_index
        self.buy_date = buy_date
        self.sell_date = None

        ## Might be worth adding a list for multiple entry points 


        self.cum_log_return = 0
        self.duration = 0

        self.is_closed = False

        def update(self, log_return):
            self.cum_log_return += log_return
            self.duration += 1
        
        def get_price(self):
            return self.buy_price * np.exp(self.cum_log_return)
        
        def close(self, sell_index, sell_date):
            self.is_closed = True
            self.sell_date = sell_date

class Strategy:
    def __init__(self, cash):
        self.cash = cash
        self.starting_cash = cash
        
        self.all_trades = defaultdict(list)
        self.open_trades = {}

    def buy(self, buy_price, buy_index, buy_date):
        if buy_price < self.cash:
            if buy_index in self.open_trades:
                current_trade = self.open_trades[buy_index]
                current_trade_equity = current_trade.get_price()
                current_trade.close(buy_index, buy_date)
                new_trade = Trade(buy_price=(buy_price+current_trade_equity), buy_index=buy_index, buy_date=buy_date)
                self.all_trades[buy_index].append(new_trade)
                self.open_trades.update({buy_index: new_trade})
                self.cash -= buy_price
            else:
                new_trade = Trade(buy_price=buy_price, buy_index=buy_index, buy_date=buy_date)
                self.all_trades[buy_index].append(new_trade)
                self.open_trades.update({buy_index: new_trade})
                self.cash -= buy_price
    
    def sell(self, sell_index, sell_date):
        if sell_index in self.open_trades:
            sold_trade = self.open_trades.pop(sell_index)
            self.cash += sold_trade.get_price()
            sold_trade.close(sell_index,sell_date)

    def run(self, row):
        pass


# BUY AND HOLD IS DIFFICULT FOR MULTIPLE STOCKS
# DOES IT EVEN HAVE VALUE?

class Backtest:
    def __init__(self, dataframe: pd.DataFrame, strategy: Strategy):
        self.dataframe = dataframe
        self.strategy = strategy
        self.spy_return = 0

    def run_backtest(self):
        for index, row in self.dataframe.iterrows():
            self.strategy.run(row)

    def visualize(self):
        pass
    # NEED TO WORK THIS OUT


#TO IMPLEMENT:
#Monte Carlo Sim
#GBM Stock Sim
#



